# 提示词结构
通用的提示词里至少包含下面这些部分：
- 指令：想要模型执行的特定任务或指令。
- 上下文：包含外部信息或额外的上下文信息，引导语言模型更好地响应。
- 例子：当用户的问题不明确的时候，可以借助举例让大模型生成类似的结果。
- 输入数据：用户输入的内容或问题。
- 输出指示：指定输出的类型或格式。

In [ ]:
### 加载模型：Qwen2.5-0.5B-Instruct模型
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_or_path = '/your/path/of/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,device_map='auto', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.17it/s]


In [2]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

In [10]:
prompt="""
指令：从以下新闻文本中提取信息
上下文：2025年7月8日，国家航天局宣布，嫦娥七号探测器计划于2026年发射，将开展月球南极环境与资源勘查。据悉，嫦娥七号将搭载4台国际合作载荷，包括与欧洲空间局合作的月表负离子分析仪。
输入问题：输出嫦娥七号的发射时间
输出：
"""
print(inference(model,tokenizer,prompt))

{'response': '嫦娥七号计划于2026年发射。', 'generated_tokens_count': 14}
